In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    database_name = '%s_upcache' % game
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{database_name}', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1640217600
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_maintenance_okr_ccl_dau_sql(game, start_time, end_time, level):
    return """SELECT main.day AS `day`,
              (SELECT SUM(prev.dau)
              FROM %s_sessions_daily_summary prev
              WHERE prev.day = main.day - 90*86400
              AND prev.townhall_level >= %d) AS `prevqtr`,
              SUM(main.dau) AS `thisqtr`
              FROM %s_sessions_daily_summary main
              WHERE main.day >= %d AND main.day < %d
              AND main.townhall_level >= %d
              GROUP BY `day` ORDER BY `day` ASC;""" % (game, level, game, start_time, end_time, level)

time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

# prepare maintenance OKR data

engine = get_engine('tr')
maintenance_okr_ccl_dau_sql = get_maintenance_okr_ccl_dau_sql('tr',start_time,end_time,5)
maintenance_okr_ccl_dau_data = pd.read_sql(maintenance_okr_ccl_dau_sql, engine)

print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

Dashboard updated Sat, 12 Mar 2022 at 16:50:12 UTC


In [2]:
# maintenance OKR display
boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries['start'] = diplay_boundaries['start'] + 86400*7
print('TR CCL5+ DAU vs. previous quarter end')
header = "{0:12} {1:8}".format('Week', 'DAU')
print(header)
for i, day in enumerate(maintenance_okr_ccl_dau_data['day']):
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    maintenance_okr_ccl_dau = maintenance_okr_ccl_dau_data['thisqtr'][i]
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(int(maintenance_okr_ccl_dau)))
        print(output_line)

TR CCL5+ DAU vs. previous quarter end
Week         DAU     
2021-12-30   2881    
2022-01-06   2942    
2022-01-13   2944    
2022-01-20   3011    
2022-01-27   3001    
2022-02-03   2913    
2022-02-10   2994    
2022-02-17   2945    
2022-02-24   2957    
2022-03-03   2923    
2022-03-10   2952    
